<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/genai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Retrieval Augmented Generation (RAG)
# ---
# In diesem Beispiel erstellen wir ein RAG-System.
# Warum ist RAG sinnvoll? Weil ein LLM nicht per se inhaltlich korrekten Inhalt
# produziert, sondern wie ein "stochastischer Papagei" entlang seiner Trainings-
# daten Tokens basierend auf vorherigen Tokens produziert.
# Wenn wir uns sicher sein wollen, dass die Inhalte korrekt sind, brauchen wir
# also andere Methoden. Und manchmal ist es ohnehin so, dass wir sehr spezielle
# Informationen verarbeiten müssen (z.B. geheime firmeninterne Dokumente), von
# denen das LLM sowieso nicht direkt etwas Bescheid weiß.
# Die Grundidee eines RAG-Systems ist, dass ein Sprachmodell bei der
# Beantwortung von Fragen externe Wissensquellen durchsucht und relevante
# Informationen in seine Antwort integriert.

In [32]:
# Wir wollen hier mit den LLMs von OpenAI arbeiten. Da wir ein Backend-System
# entwickeln, hilft uns die UI-Version von ChatGPT nicht weiter, sondern wir
# brauchen Zugriff zur Developer-API mittels API Key. Jeder von Euch hat von mir
# entweder einen API Key von meinem persönlichen Account bekommen, oder nutzt
# einen aus seinem eigenen Account. Dieser Key muss - falls Ihr in Google
# Colab bleibt - als Secret hier hinterlegt werden, ansonsten in einem lokalen
# Environment. In Google Colab müsst Ihr links auf das Schlüssel-Symbol klicken
# und den Key dort copy pasten sowie dem Key einen Namen geben. Diesen Namen
# nutzt Ihr dann hier in der Klammer:

from google.colab import userdata
OPENAI_API_KEY = userdata.get('apikey_rh')

In [ ]:
!pip install langchain_openai

In [33]:
# Nun nutzen wir ein LLM von OpenAI, indem wir die API direkt ansprechen, in
# diesem Fall das gpt-4o-mini:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [34]:
# Nun kann mit model.invoke() ein Prompt an die API geschickt werden, und das
# in model eingestellte Modell liefert die Antwort:
model.invoke("Was ist der Sinn des Lebens? Bitte fasse Dich kurz!")

AIMessage(content='Der Sinn des Lebens ist eine subjektive Frage, die für jeden individuell ist. Viele Menschen finden Erfüllung durch Beziehungen, persönliche Entwicklung, das Streben nach Glück oder das Hinterlassen eines positiven Einflusses. Letztendlich liegt es an jedem Einzelnen, seinen eigenen Sinn zu entdecken.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 19, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Brk4bVObghfRuJuwS0OwR6ogmlRAS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c97377ff-ab9f-4d04-95d4-2e3098c083b1-0', usage_metadata={'input_tokens': 19, 'output_tokens': 58, 'total_tokens': 77, 'input_token_detai

In [35]:
# Wir können nun den Parser in Langchain verwenden, um nur die Antwort des
# AIMessage-Objekts zu bekommen. Gleichzeitig nutzen wir die Verkettungs-
# funktion von Langchain über das Pipe-Symbol |, d.h. wir können eine chain
# von Arbeitsschritten bauen, in der der Output links von | der Input von rechts
# von | wird.
# In diesem Fall wollen wir, dass der Output des Modells in den Parser geht und
# dieser nur die Antwort zurückgibt.
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Was ist der Sinn des Lebens? Bitte fasse Dich kurz!")

'Der Sinn des Lebens ist eine philosophische Frage, die je nach persönlicher Überzeugung und Erfahrung unterschiedlich beantwortet wird. Viele Menschen finden Sinn in Beziehungen, persönlichem Wachstum, Liebe, Freude, einer sinnvollen Tätigkeit oder dem Streben nach Wissen. Letztlich liegt es an jedem Einzelnen, seinen eigenen Sinn zu definieren.'

In [36]:
# Nun können wir die Frage an das LLM auch besser strukturieren, indem wir
# Frage und Kontext voneinander trennen. Hierfür bietet bspw. Langchain eine
# Funktionalität namens ChatPromptTemplate an, mit der ein solches Template
# gebaut werden kann.
from langchain.prompts import ChatPromptTemplate

template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ralfs Bruder heißt Axl.", question="Wer ist Ralfs Bruder?")

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".\n\nContext: Ralfs Bruder heißt Axl.\n\nQuestion: Wer ist Ralfs Bruder?\n'

In [37]:
# Nun können wir wieder mit der Verkettungsfunktion von Langchain den Prompt
# als Input in das Modell geben, und dessen Output wiederum an den Parser.
chain = prompt | model | parser
chain.invoke({
    "context": "Ralfs Bruder heißt Axl",
    "question": "Wer ist Ralfs Bruder?"
})

'Ralfs Bruder ist Axl.'

In [38]:
# Wir können die Kette beliebig erweitern, z.B. um ein Übersetzungs-Template.
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [39]:
# Jetzt können wir sogar eine Chain innerhalb einer Chain verwenden:
# Wir verwenden die bisherige Chain ("chain"), um ihr "context" und "question"
# (auf Deutsch) zu geben. Diese wird zusammen mit der Zielsprache an den
# Übersetzungs-Promt übergeben, die dies wiederum an das Modell übergibt.
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "English",
    }
)

'Ralf has a total of three siblings.'

In [40]:
# Nun wollen wir mehr Kontext erlauben als nur zwei Sätze. Dazu verwenden wir
# in diesem Beispiel ein Transkript von einem youtube-Video, wo Andrej Karpathy
# über AI spricht:
# https://www.youtube.com/watch?v=cdiD-9MMpb0
# Das Transkript davon ist bereits erstellt und liegt in GitHub.
import requests

url = "https://raw.githubusercontent.com/RalfH1388/genai-lecture/main/data_interview.txt"
response = requests.get(url)
interview = response.text

print(interview[:100])

I think it's possible that physics has exploits and we should be trying to find them. arranging some


In [41]:
# Nun machen wir einen Schritt, der nicht empfehlenswert ist, aber trotzdem
# einmal getan werden muss, um zu zeigen, wie es NICHT geht.
# Eine naive Idee könnte jetzt sein, einfach das gesamte Textdokument mit dem
# Interview als Kontext an das LLM zu geben:
chain.invoke({
  "context": interview,
  "question": "Is reading papers a good idea?"
})

'Yes, reading papers can be a good idea, as they often provide insights into current research, methodologies, and developments in a field. They can help deepen your understanding of specific topics and keep you informed about advancements. However, it is also important to recognize that not all papers are equally valuable, and some may require careful evaluation to ascertain their relevance and quality.'

In [42]:
# Wie wir an der Antwort sehen, hat dies ganz gut funktioniert. Dies ist aber
# nicht immer so, und hat außerdem zwei wichtige Limitationen (die zugleich
# konstituierende Gründe sind für die Verwendung von RAG anstatt plain-LLM):
# - viele benötigte Kontexte (bspw. firmeninterne 100-seitige Dokumente) sind
#   für die context windows der jeweiligen LLMs zu groß, sprich: das LLM ist
#   gar nicht in der Lage, so viele Tokens auf einmal in einer Anfrage zu
#   bearbeiten (abgesehen davon, dass es kostenintensiv ist)
# - selbst wenn irgendwann einmal LLMs ausreichend hohe context windows hätten,
#   macht es immer noch wenig Sinn, zu viel Kontext mitzugeben. Wenn man einfach
#   alles mitgibt, kann das Modell den Fokus verlieren oder irrelevante Teile
#   berücksichtigen.
# Hier kommt nun die Idee von RAG in's Spiel: wir wollen nur relevanten Kontext
# als Input für das Modell verwenden. Die Frage ist nur: wie kommen wir dahin,
# a priori relevanten von irrelevantem Kontext zu unterscheiden?
# Antwort: wir unterteilen unseren Kontext in kleinere Teile (sog. Chunks),
# und wollen nun die gestellt Frage mit all diesen Chunks vergleichen, und nur
# die Chunks an das Modell geben, die am Relevantesten sind. Nun bleibt aber
# immer noch die Frage offen: wie entscheiden wir, welche Chunks am
# Relevantesten sind? Die Antwort kommt gleich (Stichwort: Embeddings), aber
# zunächst mal kümmern wir uns um das Chunking des Interviews.

In [ ]:
!pip install langchain-community

In [44]:
# Hier für gibt es wieder vordefinerte Frameworks, die den Text in etwa gleiche
# Teile aufteilen. Dazu müssen wir zunächst das Interview in die benötigten
# Strukuten laden (der unten stehende Code ist etwas hemdsärmelig, es ist nicht
# direkt wichtig ihn zu verstehen)
import requests
from langchain_community.document_loaders import TextLoader
import tempfile

# Datei herunterladen
response = requests.get(url)
response.raise_for_status()

# Temporäre Datei schreiben
with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode="w") as tmp:
    tmp.write(response.text)
    temp_path = tmp.name

# Mit TextLoader laden
loader = TextLoader(temp_path)
text_documents = loader.load()

print(text_documents[0].page_content[:500])

I think it's possible that physics has exploits and we should be trying to find them. arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These synthetic AIs will uncover that puzzle and solve it. The following is a conv


In [47]:
# Natürlich bietet Langchain nun auch ein Framework, wie man die Chunks ohne
# explizites Coding erstellen kann. In diesem Fall erstellt untenstehende
# Funktionalität Chunks, bei denen jeder Chunk in etwa 100 Zeichen hat (die
# Prozedur schneidet nicht hart beim 101. Zeichen ab, sondern versucht, Worte
# nicht auseinanderzureißen) und einen Overlapt von 20 Zeichen (d.h. ca. die
# letzten 20 Zeichen von Chunk i sind auch ca. die ersten 20 Zeichen von Chunk
# i+1). Warum? Beim Splitten kann es passieren, dass ein wichtiger Gedanke,
# Satz oder Begriff genau am Ende eines Chunks aufhört – und dann im nächsten
# Chunk außer Kontext steht. Durch Overlap wird ein Teil des vorherigen Kontexts
# in den nächsten Chunk übernommen, damit dieser "weiß", worum es vorher ging.

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': '/tmp/tmpddgnw7vd.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them. arranging some"),
 Document(metadata={'source': '/tmp/tmpddgnw7vd.txt'}, page_content='arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow,'),
 Document(metadata={'source': '/tmp/tmpddgnw7vd.txt'}, page_content='buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences'),
 Document(metadata={'source': '/tmp/tmpddgnw7vd.txt'}, page_content="intelligences are kind of like the next stage of development. And I don't know where it leads to."),
 Document(metadata={'source': '/tmp/tmpddgnw7vd.txt'}, page_content='where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These')]

In [ ]:
# Auch bei der Größe der Chungs sollte man gewisse Grenzen einhalten. Zu kleine
# Chunks (< 100 Tokens) enthalten oft zu wenig Kontext, und führen
# zu sinnlosen Embeddings (z. B. nur „Abschnitt 2.1: Einleitung“).mZu große
# Chunks (> 1000 Tokens) erschweren das Matching bei Embedding-Suche (Embeddings
# sind zu unspezifisch), können bei LLMs zu Token-Limit-Problemen führen, und
# senken die Präzision bei Kontext-Retrieval („too much noise“).

In [46]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [52]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

embedded_query = embeddings.embed_query("Wer ist Ralfs Bruder?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.024147924035787582, 0.025537647306919098, -0.020976504310965538, -0.015940243378281593, -0.034351106733083725, 0.0030080974102020264, -0.007103029638528824, 0.0172111876308918, -0.042903248220682144, -0.038057032972574234]


In [53]:
sentence1 = embeddings.embed_query("Ralfs Bruder ist Axl")
sentence2 = embeddings.embed_query("Michaelas Bruder ist Paul")

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(np.float64(0.7451138269282955), np.float64(0.4752776876358714))

In [ ]:
!pip install docarray

In [57]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Ralfs Bruder heißt Axl",
        "Michaela und Paul sind Geschwister",
        "Dennis mag weiße Autos",
        "Anna Mutter ist Lehrerin",
        "Hektor fährt einen schwarzen Audi",
        "Michaela hat zwei Geschwister",
    ],
    embedding=embeddings,
)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_IYGimIkXl9Jrvw6o15RHOQYt` does not have access to model `text-embedding-3-small`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
vectorstore1.similarity_search_with_score(query="Wer ist Ralfs Bruder?", k=3)

[(Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  np.float64(0.9128045266088952)),
 (Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
  np.float64(0.8209296823604758)),
 (Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
  np.float64(0.8029525892979162))]

In [ ]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Wer ist Ralfs Bruder?")

[Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
 Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
 Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
 Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi')]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Welche Farbe hat Hektors Auto?")

{'context': [Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi'),
  Document(metadata={}, page_content='Dennis mag weiße Autos'),
  Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  Document(metadata={}, page_content='Michaela hat zwei Geschwister')],
 'question': 'Welche Farbe hat Hektors Auto?'}

In [ ]:
chain = setup | prompt | model | parser
chain.invoke("Welche Farbe hat Hektors Auto?")

'Hektors Auto ist schwarz.'

In [ ]:
chain.invoke("Welches Auto fährt Hektor?")

'Hektor fährt einen schwarzen Audi.'

In [ ]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [ ]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence refers to advanced artificial intelligence systems that are seen as the next stage of development in AI. These systems are thought to possess capabilities that allow them to uncover and solve complex problems or puzzles within the universe. Unlike traditional forms of AI, synthetic intelligences are anticipated to engage in tasks that involve understanding emotions, creativity, and generating art and ideas autonomously.'